In [29]:
import pandas as pd

In [30]:
df=pd.read_csv("/home/jackdaniel/Documents/learnig skill/tenserflow practice/cal_housing_clean.csv")

In [31]:
df.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [32]:
x=df.drop("medianHouseValue",axis=1)
y=df["medianHouseValue"]

In [33]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=101)

## Normalize the data

In [34]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()

In [35]:
scaler.fit(x_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [36]:
x_train = pd.DataFrame(data=scaler.transform(x_train),columns = x_train.columns,index=x_train.index)

In [37]:
x_train

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
6761,0.352941,0.069688,0.117163,0.048769,0.115442,0.142508
3010,0.607843,0.011242,0.015673,0.008367,0.014142,0.045027
7812,0.666667,0.025230,0.031347,0.020971,0.030258,0.212866
8480,0.666667,0.032530,0.033830,0.024752,0.030094,0.298651
1051,0.294118,0.031919,0.035692,0.019466,0.034863,0.272631
...,...,...,...,...,...,...
5695,0.647059,0.061524,0.073557,0.043028,0.074494,0.288637
8006,0.686275,0.024264,0.024519,0.014459,0.025654,0.285720
17745,0.098039,0.089628,0.086592,0.048139,0.080579,0.389402
17931,0.686275,0.021110,0.032588,0.018976,0.034534,0.191818


In [38]:
x_test = pd.DataFrame(data=scaler.transform(x_test),columns = x_test.columns,index=x_test.index)

In [39]:
import tensorflow as tf

In [40]:
df.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [41]:
hm=tf.feature_column.numeric_column("housingMedianAge")
tr=tf.feature_column.numeric_column("totalRooms")
tbr=tf.feature_column.numeric_column("totalBedrooms")
p=tf.feature_column.numeric_column("population")
h=tf.feature_column.numeric_column("households")
mi=tf.feature_column.numeric_column("medianIncome")

In [42]:
feat_cols=[hm,tr,tbr,p,h,mi]

## Estimator

In [43]:
model = tf.estimator.DNNRegressor(hidden_units=[10,10,10],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp584g6_14', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa58abcc780>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Input function

In [44]:
input_fn=tf.estimator.inputs.pandas_input_fn(x=x_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

## Train the model

In [46]:
model.train(input_fn,steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp584g6_14/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1000 into /tmp/tmp584g6_14/model.ckpt.
INFO:tensorflow:loss = 98409415000.0, step = 1001
INFO:tensorflow:global_step/sec: 152.477
INFO:tensorflow:loss = 53803536000.0, step = 1101 (0.667 sec)
INFO:tensorflow:global_step/sec: 104.772
INFO:tensorflow:loss = 145202430000.0, step = 1201 (0.945 sec)
INFO:tensorflow:global_step/sec: 192.654
INFO:tensorflow:loss = 23581846000.0, step = 1301 (0.519 sec)
INFO:tensorflow:global_step/sec: 198.593
INFO:tensorflow:loss = 208761080000.0, step = 1401 (0.503 sec)
INFO:tensorflow:global_step/sec: 185.656
INFO:tensorflow:loss = 306256670000.0, step = 1501 (0.545 sec)
INFO:tensorflow:global_step/sec: 183.071
I

## Evaluation

In [21]:
test_input=tf.estimator.inputs.pandas_input_fn(x=x_test,y=y_test,batch_size=10,num_epochs=1,shuffle=False)

In [22]:
result=model.evaluate(test_input)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-12T09:54:17Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp6pd1rurn/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-05-12-09:54:21
INFO:tensorflow:Saving dict for global step 2000: average_loss = 11029807000.0, global_step = 2000, label/mean = 207662.89, loss = 110155750000.0, prediction/mean = 208243.2
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: /tmp/tmp6pd1rurn/model.ckpt-2000


In [23]:
result

{'average_loss': 11029807000.0,
 'label/mean': 207662.89,
 'loss': 110155750000.0,
 'prediction/mean': 208243.2,
 'global_step': 2000}

## Predicction

In [24]:
pred_input=tf.estimator.inputs.pandas_input_fn(x=x_test,batch_size=10,num_epochs=1,shuffle=False)

In [25]:
prediction=model.predict(pred_input)

In [26]:
prd=list(prediction)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp6pd1rurn/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [27]:
prd

[{'predictions': array([224849.06], dtype=float32)},
 {'predictions': array([267322.44], dtype=float32)},
 {'predictions': array([218581.53], dtype=float32)},
 {'predictions': array([179788.75], dtype=float32)},
 {'predictions': array([245005.47], dtype=float32)},
 {'predictions': array([202934.53], dtype=float32)},
 {'predictions': array([228544.1], dtype=float32)},
 {'predictions': array([205538.75], dtype=float32)},
 {'predictions': array([206705.9], dtype=float32)},
 {'predictions': array([167609.9], dtype=float32)},
 {'predictions': array([206323.53], dtype=float32)},
 {'predictions': array([229834.53], dtype=float32)},
 {'predictions': array([192526.62], dtype=float32)},
 {'predictions': array([176031.89], dtype=float32)},
 {'predictions': array([254837.94], dtype=float32)},
 {'predictions': array([170498.9], dtype=float32)},
 {'predictions': array([202472.7], dtype=float32)},
 {'predictions': array([183168.61], dtype=float32)},
 {'predictions': array([179434.2], dtype=float32)},